<a href="https://colab.research.google.com/github/hikmet96/play_generator_NLP_RNN/blob/main/play_generator_NLP_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNN Play Generator

In [ ]:
%tensorflow_version 2.x
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


# Dataset

In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


# Loading data

In [ ]:
# from google.colab import files
# path_to_file = list(files.upload().keys())[0]

# Reading contents of file

In [ ]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
#length of this text is the number of characters in it
print('Length of text :{} characters'.format(len(text)))

Length of text :1115394 characters


In [ ]:
# Look at first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



# Encoding
Encoding each characters as different integers

In [ ]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [ ]:
# Look at our encoded text and how it is encoded
print("Text:" , text[:13])
print("Encoded text:", text_to_int(text[:13]))

Text: First Citizen
Encoded text: [18 47 56 57 58  1 15 47 58 47 64 43 52]


and here a function that convert int to text

In [ ]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


# Creating training examples

In [ ]:
seq_length = 100 # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

Next we can use the batch method to turn this stream of characters into batches of desired length.

In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

Now  we need to use these sequences of length 101 and split them into input and output.

In [ ]:
def split_input_target(chunk): # for the example: hello
    input_text = chunk[:-1] # hell
    target_text = chunk[1:] # hell, ello
    return input_text, target_text # hell, ello

dataset = sequences.map(split_input_target) # we use map to apply the above function to every entry

In [ ]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


Finally we need to make training batches

In [ ]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab) # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# si ot dosen't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements)
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# Building the model
Now it is time to build the model. We will use an embedding layer a LSTM and one dense layer that contains a node for each unique character in our training data. The dense layer will give us a probability distribution over all nodes

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5330241 (20.33 MB)
Trainable params: 5330241 (20.33 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


###Creating a Loss Function
Now we are going to create our own loss function for this problem. This is because our model will output a (64, sequence_length, 65) shaped tensor that represents the probability distribution of each character at each timestep for every sequence in the batch.


In [ ]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [ ]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-3.1286671e-03  5.3959235e-04 -4.8131845e-03 ...  6.1494159e-03
    3.3795787e-04 -4.5271297e-03]
  [ 3.0494318e-03 -1.6834310e-03 -3.1218315e-03 ...  7.6083196e-03
    6.4839644e-04 -7.4941581e-03]
  [ 3.4384406e-04 -5.5479310e-03 -2.9910440e-03 ...  6.2410515e-03
    4.3832706e-03 -3.5976488e-03]
  ...
  [ 1.3252220e-03 -1.5116751e-03 -5.2458989e-03 ... -9.6084730e-04
    8.8310149e-03  4.0338619e-04]
  [ 2.3593274e-03 -1.2010744e-02 -7.1263225e-03 ... -3.0092071e-03
    3.8868831e-03  2.4625543e-03]
  [-1.7483600e-03 -1.5782572e-02 -7.2499653e-03 ... -1.6045007e-03
    7.2209444e-03  5.4652686e-03]]

 [[ 5.1802257e-03 -2.6013600e-03 -7.2677678e-04 ...  3.1607980e-03
    9.9433435e-04 -3.7844472e-03]
  [ 8.5301674e-04 -9.4504096e-04 -5.7362090e-03 ...  8.0027860e-03
    5.9823203e-04 -8.2279798e-03]
  [-3.5276476e-03 -3.1785234e-03  9.1245375e-04 ...  5.0999257e-03
    1.4623690e-03 -4.3325541e-03]
  ...
  [-4.8530749e-03  4.1589625e-03 -4.6657813e-03 ...  9.5338672e

In [ ]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[-0.00312867  0.00053959 -0.00481318 ...  0.00614942  0.00033796
  -0.00452713]
 [ 0.00304943 -0.00168343 -0.00312183 ...  0.00760832  0.0006484
  -0.00749416]
 [ 0.00034384 -0.00554793 -0.00299104 ...  0.00624105  0.00438327
  -0.00359765]
 ...
 [ 0.00132522 -0.00151168 -0.0052459  ... -0.00096085  0.00883101
   0.00040339]
 [ 0.00235933 -0.01201074 -0.00712632 ... -0.00300921  0.00388688
   0.00246255]
 [-0.00174836 -0.01578257 -0.00724997 ... -0.0016045   0.00722094
   0.00546527]], shape=(100, 65), dtype=float32)


In [ ]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[-0.00312867  0.00053959 -0.00481318  0.00323856 -0.00592589 -0.0045141
 -0.00044255  0.0009917  -0.00134415 -0.00219801 -0.00252742 -0.0049194
 -0.00397376 -0.00014339 -0.00129214  0.00100012  0.00556242  0.00543164
  0.0043218   0.00207613 -0.00294037  0.00341148  0.00040095 -0.00327952
  0.00080758 -0.00364453  0.00580798 -0.00241375 -0.00225753  0.00642638
  0.00488473  0.00290598 -0.0005871   0.0026784  -0.00279071  0.00562556
 -0.00305777 -0.00078597  0.00347138 -0.0004548  -0.00370626 -0.00301317
 -0.00327308  0.00827662  0.00436474  0.00375455  0.0041587  -0.00600551
 -0.00177978 -0.00667762  0.00165898 -0.00193441 -0.00228899  0.00555616
  0.00290092  0.0078944   0.00331845 -0.00360496  0.00028539  0.00231193
 -0.00558532  0.00457457  0.00614942  0.00033796 -0.00452713], shape=(65,), dtype=float32)


In [ ]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array *and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

"3UXH'OdrjIrn.LfvF.h&ZMSJmwFHERhGeRlfW;EoTK'$ywSo jCkqTL wLql:?&k!rz:?YSQlF?t!ctKfRRQkT:NHtVPkFPdVqk "

So now we need to create a loss function that can compare that output to the expected output and give us some numeric value representing how close the two were.


In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

###Compiling the Model
At this point we can think of our problem as a classification problem where the model predicts the probabillity of each unique letter coming next.


In [ ]:
model.compile(optimizer='adam', loss=loss)

###Creating Checkpoints
Now we are going to setup and configure our model to save checkpoinst as it trains. This will allow us to load our model from a checkpoint and continue training it.

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

###Training
Finally, we will start training the model.

**If this is taking a while go to Runtime > Change Runtime Type and choose "GPU" under hardware accelerator.**



In [ ]:
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
172/172 [==============================] - 16s 69ms/step - loss: 2.5351
Epoch 2/50
172/172 [==============================] - 13s 67ms/step - loss: 1.8474
Epoch 3/50
172/172 [==============================] - 14s 69ms/step - loss: 1.6124
Epoch 4/50
172/172 [==============================] - 14s 72ms/step - loss: 1.4867
Epoch 5/50
172/172 [==============================] - 14s 75ms/step - loss: 1.4101
Epoch 6/50
172/172 [==============================] - 15s 77ms/step - loss: 1.3550
Epoch 7/50
172/172 [==============================] - 15s 74ms/step - loss: 1.3117
Epoch 8/50
172/172 [==============================] - 14s 73ms/step - loss: 1.2721
Epoch 9/50
172/172 [==============================] - 14s 73ms/step - loss: 1.2359
Epoch 10/50
172/172 [==============================] - 14s 73ms/step - loss: 1.2002
Epoch 11/50
172/172 [==============================] - 15s 76ms/step - loss: 1.1643
Epoch 12/50
172/172 [==============================] - 15s 75ms/step - loss: 1.1275
E

###Loading the Model
We'll rebuild the model from a checkpoint using a batch_size of 1 so that we can feed one peice of text to the model and have it make a prediction.

In [ ]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

Once the model is finished training, we can find the **lastest checkpoint** that stores the models weights using the following line.



In [ ]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

We can load **any checkpoint** we want by specifying the exact file to load.

In [ ]:
checkpoint_num = 10
model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
model.build(tf.TensorShape([1, None]))

AttributeError: 'tensorflow.python.util._pywrap_checkpoint_reader.C' object has no attribute 'endswith'

###Generating Text
Now we can use the lovely function provided by tensorflow to generate some text using any starting string we'd like.

In [ ]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension

      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: romeo
romeon. Yea, let me have
you fear 'gm know.

VOLUMNIA:
Princeff from thy mother, for this prince's delicate
To your office that I have tormot me are woo for you; pray you,
Both give and knack, when it will re enture for our hearts, he knows no more of mine,
Than Antium.

DUKE VINCENTIO:
Hold you inform: my counsel's judgment,
Not doing that the nature of his trumpet:
O, sir, it impose both withal.

PETRUCHIO:
Where be the grace of hid?

Provost:
That, my lord:

KING HENRY VI:
Ay, and for this beg peace in many's eyes,
To working go our tent; the more eyes of the old man hate:
Come, come, peace still and bed, and bid our state
Abroad to lightly therefore I the duke--
Dost thou desire
To have a wise are open, and love
Lound up to flatter them
in place, and let the direful plight teer grief;
O, ne
